In [1]:
import json
import numpy as np
from tqdm import tqdm

json_file_base = '/home/erik/raid/eval_mtr/MTR_wo_poses_8a100_single.json'
json_file_att = '/home/erik/raid/eval_mtr/MTR_MLP_Attention_8a100_single.json'

with open(json_file_base) as f:
    data_base = json.load(f)
with open(json_file_att) as f:
    data_att = json.load(f)
#diff_mlp_ade_ped = list()
diff_att_ade_ped = list()
#diff_mlp_ade_cyc = list()
diff_att_ade_cyc = list()
#diff_mlp_fde_ped = list()
diff_att_fde_ped = list()
#diff_mlp_fde_cyc = list()
diff_att_fde_cyc = list()
key_list = list()
metrics = ["minADE - PEDESTRIAN", "minADE - CYCLIST", "minFDE - PEDESTRIAN", "minFDE - CYCLIST"]
lists = [diff_att_ade_ped, diff_att_ade_cyc, diff_att_fde_ped, diff_att_fde_cyc]
data_base = {key: value for d in data_base for key, value in d.items()}
data_att = {key: value for d in data_att for key, value in d.items()}
for key in tqdm(list(data_base.keys())):
    if key not in data_att.keys():
        continue
    key_list.append(key)
    for metric, list in zip(metrics, lists):
        if data_base[key][metric] > 0.0:
            list.append(data_base[key][metric] - data_att[key][metric])
        else:
            list.append(-np.inf)

paired_lists = sorted(zip(lists[0], key_list), key=lambda x: x[0], reverse=True)
    
# Unzip the paired list into two lists
ade_diff_ped_att, key_list = zip(*paired_lists)
print(ade_diff_ped_att[:1000])
print(key_list[:1000])


100%|██████████| 4710/4710 [00:00<00:00, 1021572.65it/s]

(1.7012634376684825, 1.4236121599872906, 1.1955511569976807, 1.1311817988753319, 0.9268091320991516, 0.8351206382115682, 0.8006169795989989, 0.6272273461023965, 0.5528547565142312, 0.5472359756628672, 0.5378295133511226, 0.5158980290095012, 0.4951263268788656, 0.4857274492581686, 0.4709785580635072, 0.4451494018236796, 0.4447310591737429, 0.441041628519694, 0.44103441635767626, 0.43177667260169983, 0.42705237368742627, 0.4219477574030558, 0.41735844314098364, 0.41260250161091494, 0.40029654900232947, 0.3934479157129923, 0.38634726653496426, 0.38404623667399096, 0.3814101815223694, 0.37866107126077014, 0.36724301427602773, 0.36334459483623505, 0.3621786832809449, 0.3607016503810882, 0.35980221132437384, 0.3577919900417327, 0.35738190015157056, 0.35306693613529205, 0.34888509412606555, 0.3420534233252208, 0.3403618360559146, 0.3383692850669225, 0.3296673235793909, 0.329249918460846, 0.3239718874295552, 0.3226531396309535, 0.3200100163618723, 0.3173432151476542, 0.31559347112973535, 0.313

In [3]:
print(data_base['605c519944f01baf']["minADE - PEDESTRIAN"])
print(data_att['605c519944f01baf']["minADE - PEDESTRIAN"])

2.71210777759552
0.7464454447229704


In [21]:
import plotly.graph_objects as go
import glob
import os
import pickle
from waymo_open_dataset.utils import keypoint_draw, frame_utils
from waymo_open_dataset.utils.keypoint_draw import Wireframe
import numpy as np
from plotly.subplots import make_subplots


In [30]:
def draw_skeleton(joints, lines, fig):
    for line in lines:
        if line == 0:
            connection_point = (joints[lines[line][0]] + joints[lines[line][1]]) / 2
            fig.add_trace(go.Scatter3d(x=[joints[line][0], connection_point[0]], y=[joints[line][1], connection_point[1]], z=[joints[line][2], connection_point[2]], mode='lines', line=dict(color='black', width=2)))
        else:
            for point in lines[line]:
                fig.add_trace(go.Scatter3d(x=[joints[line][0], joints[point][0]], y=[joints[line][1], joints[point][1]], z=[joints[line][2], joints[point][2]], mode='lines', line=dict(color='black', width=2)))

In [31]:
JOINT_NAMES = ["NOSE", "LEFT_SHOULDER", "LEFT_ELBOW",
               "LEFT_WRIST", "LEFT_HIP", "LEFT_KNEE",
               "LEFT_ANKLE", "RIGHT_SHOULDER", "RIGHT_ELBOW",
               "RIGHT_WRIST", "RIGHT_HIP", "RIGHT_KNEE", "RIGHT_ANKLE"]
lines = {0: [1, 7], 1: [2, 4, 7], 2: [3], 4: [5, 10], 5: [6], 7: [8, 10], 8:[9], 10:[11], 11: [12]}
path_to_womd = '/home/erik/raid/datasets/womd/'
key = 'bf600888a4845af5' #bf600888a4845af5 (3200), 26f04b1e714986cc (961), a4e1a96c2b311cef (684), 1f70481472b7def (1551, 1554), 47f813ad7de048b8 (88)
obj_id = 3200
path = os.path.join(path_to_womd, 'lidar_snippets', key)
#print(path)
snippets = sorted(glob.glob(os.path.join(path, str(obj_id) + '_*')))
example_file = '/home/erik/raid/datasets/womd/processed_scenarios_validation/sample_' + key + '.pkl'
#print(example_file)
with open(example_file, 'rb') as f:
    example = pickle.load(f)
#print(example['track_infos']['object_id'])
index = example['track_infos']['object_id'].index(obj_id)
pos = example['track_infos']['trajs'][index, :, :3]
pose_path = os.path.join(path_to_womd, 'poses', key, 'pose_data.npy')
#print(pose_path)
pose_file = np.load(pose_path)
#print(pose_file[index])
plot = {0: (1, 1), 1: (2, 1), 2: (3, 1), 3: (4, 1), 4: (5, 1), 5: (6, 1), 6: (7, 1), 7: (8, 1), 8: (9, 1), 9: (10, 1), 10: (11, 1)} 
fig = make_subplots(
    rows=11, 
    cols=1,
    start_cell="top-left", 
    specs=[
        [{"type": "scatter3d"}], 
        [{"type": "scatter3d"}],
        [{"type": "scatter3d"}],
        [{"type": "scatter3d"}],
        [{"type": "scatter3d"}],
        [{"type": "scatter3d"}],
        [{"type": "scatter3d"}],
        [{"type": "scatter3d"}],
        [{"type": "scatter3d"}],
        [{"type": "scatter3d"}],
        [{"type": "scatter3d"}]

    ]
)
for i, snippet in enumerate(snippets):
    fig = go.Figure()
    #row, col = plot[i]
    snippet = np.load(snippet)
    snippet = snippet - pos[i]
    fig.add_trace(go.Scatter3d(x=snippet[:,0], y=snippet[:,1], z=snippet[:,2], mode='markers', marker=dict(size=2, opacity=0.8)))
    fig.add_trace(go.Scatter3d(x=pose_file[index, i, :, 0], y=pose_file[index, i, :, 1], z=pose_file[index, i, :, 2], mode='markers', marker=dict(size=4, opacity=0.8, color='red')))
    draw_skeleton(pose_file[index, i], lines, fig)
    #go.Scatter3d(x=pose_file[index, i, :, 0], y=pose_file[index, i, :, 1], z=pose_file[index, i, :, 2], mode='markers', marker=dict(size=2, opacity=0.8, color='red'))
    fig.show()